# Data-Driven Decks

Done properly, PDF will. Let's explore interactively building PDFs. 

## Before we begin

> ## This is **NOT READY** for prime-time
> Start the **demo** export server. It will try to install its dependencies with `jlpm`, and requires `nodejs`.
> ```bash
> !python scripts/drawio_export_demo.py
> ```

In [ ]:
import base64, html, urllib.parse, pathlib as P, tempfile, requests, IPython.display as D, networkx as nx, jinja2, babel.support as B, pandas as pd, PyPDF2, traitlets as T, ipywidgets as W, tornado.ioloop
from nbconvert.filters.markdown_mistune import markdown2html_mistune
from dataclasses import dataclass
import httpx

In [ ]:
class DrawioSlides(W.HTML):
    """ some number of slides, as PDF
    """
    xml = T.Unicode()
    pdf = T.Unicode()
    params = T.Dict()
    url = T.Unicode(default_value="http://localhost:8000")
    
    CORE_PARAMS = dict(
        format="pdf",
        base64="1"
    )
    
    async def update_pdf(self):
        async with httpx.AsyncClient() as client:
            xml = self.transform(self.xml)
            r = await client.post(
                self.url, 
                timeout=None,
                data=dict(
                    xml=xml, 
                    **self.params, 
                    **self.CORE_PARAMS
                ))
            self.pdf = r.text
    
    def transform(self, xml):
        return xml

    @T.observe("xml")
    def _on_xml(self, change=None):
        tornado.ioloop.IOLoop.current().add_callback(self.update_pdf)
    
    @T.observe("pdf")
    def _on_pdf(self, change=None):
        self.value = f"""
            <iframe src="data:application/pdf;base64,{self.pdf}" width="100%" height="600px" style="height: 100%"></iframe>
        """

In [ ]:
slides = DrawioSlides(xml=P.Path("testfiles/template deck.dio").read_text())
slides

In [ ]:
class TemplatedDrawioSlides(DrawioSlides):
    context = T.Dict()
    
    AMP = "&"
    _AMP_ = "_____AMP_____"

    def clean(self, txt):
        return txt.replace(self.AMP, self._AMP_)

    def smudge(self, txt):
        return txt.replace(self._AMP_, self.AMP)
    
    def transform(self, xml):
        env = jinja2.Environment(
          extensions=['jinja2.ext.i18n', 'jinja2.ext.autoescape'],
          autoescape=jinja2.select_autoescape(['html', 'xml'])
        )

        return self.smudge(env.from_string(self.clean(xml)).render(**{
            key: self.markdown(value)
            for key, value in (self.context or {}).items()
        }))
    
    @T.observe("context")
    def _on_context(self, change):
        self._on_xml()
    
    def markdown(self, md):
        return markdown2html_mistune(md).replace(self.AMP, self._AMP_)

In [ ]:
slides = TemplatedDrawioSlides(
    xml=P.Path("testfiles/template deck.dio").read_text()
)
slides

In [ ]:
slides.context = {"title": "This is the _TITLE_"}